In [1]:
import os
import csv
import pandas as pd
import numpy as np
from tools.segmentation.stats import *
import matplotlib.pyplot as plt
import seaborn as sns
from mrcnn import visualize
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

from dicom_to_cnn.tools.cleaning_dicom.folders import * 

In [2]:
csv_pred_path = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/SEGMENTATION/SEG_NIFTI_PATH_V6.csv'
df = pd.read_csv(csv_pred_path)
df.head()

,PATIENT_ID,PET,STUDY_UID,NIFTI_CT,NIFTI_PET,NIFTI_MASK,STUDY
0,13011101331015,pet0,1.2.124.113532.10.149.172.5.20131107.93455.772...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,ahl
1,13011101051001,pet0,1.2.124.113532.172.16.25.2.20110720.111654.943617,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,ahl
2,13011101431006,pet0,1.2.124.113532.172.16.25.2.20120125.456.1038547,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,ahl
3,13011101431010,pet0,1.2.124.113532.172.16.25.2.20120605.739.1104125,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,ahl
4,13011101431009,pet0,1.2.124.113532.172.16.25.2.20120606.612.1104622,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,/media/oncopole/d508267f-cc7d-45e2-ae24-9456e0...,ahl


In [4]:
study = []
number = []
for idx,name in enumerate(df['STUDY'].value_counts().index.tolist()):
    #print('Study:', name)
    study.append(name)
    #print('Number :', df['study'].value_counts()[idx])
    number.append(df['STUDY'].value_counts()[idx])
print(study)
print(number)

percent = []
for s, n in zip(study, number):
    print(s)
    print("{} % ".format(round(n/np.sum(number)*100,2)))
    print('')
    percent.append(round(n/np.sum(number)*100,2))

['ahl', 'gained', 'relevance', 'remarc', 'flip', 'lnh073b']
[703, 573, 277, 217, 125, 61]
ahl
35.94 % 

gained
29.29 % 

relevance
14.16 % 

remarc
11.09 % 

flip
6.39 % 

lnh073b
3.12 % 



In [9]:
import csv
path = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/PVAB_NIFTI/PVAB_NIFTI_V2.csv'
with open(path, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar=',')
    dataset = []
    for row in reader:
        dataset.append(row)

del dataset[0]

In [10]:
dataset[0]

['13014101023001',
 '13014101023001',
 '1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715',
 'PET0',
 '/media/oncopole/LACIE SHARE/PVAB/NIFTI/13014101023001/PET0/1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715_nifti_CT.nii',
 '/media/oncopole/LACIE SHARE/PVAB/NIFTI/13014101023001/PET0/1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715_nifti_PT.nii',
 '/media/oncopole/LACIE SHARE/PVAB/NIFTI/13014101023001/PET0/1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715_nifti_mask.nii']

In [11]:
for row in dataset : 
    for i in range(4,7):
        x = row[i].replace('/media/oncopole/LACIE SHARE/PVAB/NIFTI', '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/PVAB_NIFTI')
        row[i] = x

In [12]:
dataset[0]

['13014101023001',
 '13014101023001',
 '1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715',
 'PET0',
 '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/PVAB_NIFTI/13014101023001/PET0/1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715_nifti_CT.nii',
 '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/PVAB_NIFTI/13014101023001/PET0/1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715_nifti_PT.nii',
 '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/PVAB_NIFTI/13014101023001/PET0/1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715_nifti_mask.nii']

In [13]:
directory = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/NIFTI/PVAB_NIFTI'
filename = 'PVAB_NIFTI_V3.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT_ID", "PET", "STUDY_UID", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK", "STUDY"])
    for serie in dataset: 
        #serie = [path_1, modal_1, path_2, modal_2, study_uid, type_, patient_id, nifti_CT, nifti_PET, nifti_MASK]
        csv_writer.writerow([serie[1], serie[3], serie[2], serie[-3], serie[-2], serie[-1], 'pvab'])